<a href="https://colab.research.google.com/github/sanjaybora04/CustomAiAssistant/blob/main/nike_masked_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Required packages
- [PyTorch](https://pytorch.org/get-started/locally/): Deep learning framework.
- Hugging Face Transformers(https://huggingface.co/docs/transformers/installation): To use pretrained state-of-the-art models.
- [Hugging Face Datasets](https://pypi.org/project/datasets/): Use public Hugging Face datasets
- [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html): For interactive notebook widgets

In [1]:
!git clone https://github.com/uptrain-ai/uptrain
!mv uptrain/examples/llm_bert/* /content/
!rm -rf uptrain

Cloning into 'uptrain'...
remote: Enumerating objects: 1513, done.
remote: Counting objects: 100% (644/644), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 1513 (delta 404), reused 405 (delta 267), pack-reused 869
Receiving objects: 100% (1513/1513), 2.74 MiB | 29.82 MiB/s, done.
Resolving deltas: 100% (844/844), done.


In [2]:
!pip install torch transformers[torch] datasets ipywidgets nltk uptrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 

In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import uptrain

from model_constants import *
from model_train import retrain_model
from helper_funcs import *

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Define few cases to test our model performance before and after retraining.

In [8]:
testing_texts = [
    "Nike shoes are very [MASK].",
    "Nike shoes are good for [MASK]",
    "Nike shoes make me look [MASK]",
    "Nike's all products are [MASK]",
    "Nike"
]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
original_model_outputs = [test_model(model, x) for x in testing_texts]

Let's use Nike onlinestore customer reviews from Kaggle and filter data using UpTrain signals to retrain our model upon. Please download the data from the [link](https://www.kaggle.com/datasets/tinkuzp23/nike-onlinestore-customer-reviews?resource=download) and unzip it here.
  

In [10]:
from google.colab import files

files.upload()

Saving web_scraped.csv to web_scraped.csv


{'web_scraped.csv': b'Reviewer,Title,Content,Star_rating,Rating,Date,Reviews_posted,Location\r\nLisa Stammer,Easy website,"Easy website, fast shipping,  wonderful customer service.  Great experience",4, Great,2020-12-29,5,United States\r\nsharon richards,Hey Merry Christmas Nike!!,Hey Merry Christmas Nike!!Delivery scheduled for 22nd ha ha!!Still not with DPD as goods being sent from Belgium - so I would hazard a guess at them being sat in some port now not able to come to the UK!!No communications - except I will issue a refund- errr no thanks I want the goods I have purchased- that you have the funds for !!,1,Bad,2020-12-23,2,United Kingdom\r\njason,Nike along with apple and coca cola supporting slavery in china,"""I read in the New York Times that Nike is fighting the passage of the Uighur Forced Labor Prevention Act, a bill that would ensure that forced Uighur labor isn\'t used in the making of parts you buy. That\'s gross, and I will absolutely stop buying your products if you con

In [11]:
# Create Nike review training dataset
nike_attrs = {
    "version": "0.1.0",
    'source': "nike review dataset",
    'url': 'https://www.kaggle.com/datasets/tinkuzp23/nike-onlinestore-customer-reviews?resource=download',
}
# Download the dataset from the url, zip it and copy the csv file here
nike_reviews_dataset = create_dataset_from_csv("web_scraped.csv", "Content", "nike_reviews_data.json")

In [12]:
def nike_positive_sentiment_func(inputs, outputs, gts=None, extra_args={}):
    is_positives = []
    for input in inputs["text"]:
        txt = input.lower()
        sia = SentimentIntensityAnalyzer()
        score = sia.polarity_scores(txt)

        is_negative = score['pos'] < 0.25
        for neg_adj in ['expensive', 'worn', 'cheap', 'inexpensive', 'dirty', 'bad']:
            if neg_adj in txt:
                is_negative = True

        is_positives.append(bool(1-is_negative))
    return is_positives

cfg = {
    'checks': [{
        'type': uptrain.Anomaly.EDGE_CASE,
        "signal_formulae": uptrain.Signal("Nike Positive Sentiment", nike_positive_sentiment_func)
    }],

    # Define where to save the retraining dataset
    'retraining_folder': "uptrain_smart_data",
    
    # Define when to retrain, define a large number because we are using UpTrain just to create retraining dataset
    'retrain_after': 10000000000
}

framework = uptrain.Framework(cfg)

In [13]:
with open(nike_reviews_dataset) as f:
    all_data = json.load(f)

for sample in all_data['data']:
    inputs = {'data': {'text': [sample['text']]}}
    framework.log(inputs = inputs, outputs = None)

50  edge cases identified out of  135  total samples


In [14]:
print("Number of samples filtered for retraining: ", len(pd.read_csv("uptrain_smart_data/1/smart_data.csv")))
retraining_dataset = create_dataset_from_csv("uptrain_smart_data/1/smart_data.csv", "text", "retrain_dataset.json", min_samples=1000)

Number of samples filtered for retraining:  82


In [15]:
retrain_model(model, retraining_dataset)
retrained_model_outputs = [test_model(model, x) for x in testing_texts]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-612e13e7cb738a7e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 204
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Number of trainable parameters = 66985530


>>>Before training, Perplexity: 66.00


Epoch,Training Loss,Validation Loss
1,No log,3.676466
2,No log,3.352248
3,No log,3.394145


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 23
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 23
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 23
  Batch size = 64


Training completed. Do not forget to shar

>>>After training, Perplexity: 36.13


In [16]:
[original_model_outputs, retrained_model_outputs]

[[['popular', 'expensive', 'durable', 'common', 'comfortable'],
  ['.', 'walking', ';', 'wear', 'nike'],
  ['sexy', '.', 'pretty', 'cute', 'ridiculous']],
 [['popular', 'expensive', 'durable', 'comfortable', 'common'],
  ['.', ';', 'walking', 'nike', ':'],
  ['.', 'sexy', 'pretty', 'better', 'good']]]